# Evaluating Legal Models

LegalEase: Nick, Ben, Lisa, Devin

# Setup
### Install and import necessary packages:


In [4]:
# Installs
!pip install google-generativeai
!pip install --upgrade transformers
!pip install torch
!pip install peft
!pip install bitsandbytes
!pip install langchain-community
!pip install langchain-Chroma
!pip install accelerate
!pip install optimum
!pip install gptqmodel
!pip install auto-gptq
!pip install gpt4all

  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.30.0 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.6 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.3 which is incompatible.
google-cloud-translate 3.15.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.3 which is incompatible.
proto-plus 1.23.0 requires protobuf<5.0.0dev,>=3.19.0, but you have protobuf 5.29.3 which is incompatible.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


  Using cached gptqmodel-1.9.0.tar.gz (213 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [22 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\nicho\AppData\Local\Temp\pip-install-kzq6unq5\gptqmodel_a4b1b9d19ea340c1a5d99bfb63c4a3a4\setup.py", line 129, in <module>
      import torch  # noqa: E402
      ^^^^^^^^^^^^
    File "C:\Users\nicho\anaconda3\Lib\site-packages\torch\__init__.py", line 1836, in <module>
      _logging._init_logs()
    File "C:\Users\nicho\anaconda3\Lib\site-packages\torch\_logging\_internal.py", line 712, in _init_logs
      _update_log_state_from_env()
    File "C:\Users\nicho\anaconda3\Lib\site-packages\torch\_logging\_internal.py", line 608, in _update_log_state_from_env
      log_state = _parse_log_settings(log_setting)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\nicho\anaconda3\Lib\site-pa

In [5]:
# Imports
import pandas as pd
import time
import numpy as np
import json
import random
import os
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from huggingface_hub import login

In [6]:
# Our modules
from model import generate_response, init_model
import rag
import chat_history

### Gather resources:

In [7]:
# this is used if you want to integrate gemini
def prepare_google_env():
# Gathering Google resources
    with open('secrets.json', 'r') as file:
        secrets = json.load(file)

    import google.generativeai as genai  
    GOOGLE_API_KEY = secrets['GOOGLE_API_KEY']
    genai.configure(api_key = GOOGLE_API_KEY)
#prepare_google_env()

# Example Q_A and Converse

In [ ]:
# Only uncomment if you want to see how the Q_A script currently runs.
#import Q_A

In [10]:
import converse

here


FileNotFoundError: [Errno 2] No such file or directory: '/dev/null'

# Load Gemini

SOTA model that will be our benchmark to evaluate general response performace against. 

In [ ]:
def query_gemini(user_input, use_rag, prompt):
    """
    user_input is the input to the llm.
    Rag is a boolean if you want RAG to be used to enhance query or not
    returns a response text
    """
    rag_context = ''
    if use_rag:
        rag_context, sources = rag.query_rag(user_input)
    chat_history = ''
    user_input_with_context = prompt.format(user_input=user_input, rag_context=rag_context, chat_history=chat_history)
    
    model = genai.GenerativeModel('gemini-1.5-flash',system_instruction=user_input_with_context)
    chat = model.start_chat()
    response = chat.send_message(user_input)
    return response.text.strip()

# Load Our Model
Mistral 7B model trained on QLora Q/A Pairs.

In [ ]:
with open("prompt.txt", "r") as file:
    prompt = file.read()
model, tokenizer = init_model()
with open('prompt_short.txt', 'r') as file:
    prompt_short  = file.read()


def query_model(user_input, use_rag, prompt):
    """
    query is the input to the llm.
    Rag is a boolean if you want RAG to be used to enhance query or not
    returns a response text
    """
    rag_context = ''
    if use_rag:
        rag_context, sources = rag.query_rag(user_input)
    chat_history = ''
    user_input_with_context = prompt.format(user_input=user_input, rag_context=rag_context, chat_history=chat_history)
    new_input = 'For Context:' + rag_context + "Answer the following:" + user_input
    llm_output = generate_response(user_input_with_context, model, tokenizer)
    return llm_output

In [ ]:
def run_through_questions(model_type, use_rag, prompt):
    # Runs given model through 100 questions from the set
    # returns the average cosine similarity
    scores = []
    content_sim = []
    # Select first 100 pairs
    qa_split_100 = qa_split[:100] 

    for pair in qa_split_100:
        query = pair[0]
        answer = pair[1]
        response = ''
        
        if model_type == 'g':
            response = query_gemini(query, use_rag, prompt)
            # We will be querying twice in this script, and cannot exceed 15 a minute.
            time.sleep(8.01)
        else:
            response = query_model(query, use_rag, prompt)
            r = response.split('**Response**')
            if len(r) > 1:
                response = r[1]
        
        # Vectorize the query and response
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([query, response])

        # Compute the cosine similarity between query and response
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
        
        # Append the cosine similarity score to scores
        scores.append(similarity[0][0])
        
        # use Geval to determine if answer/response are similar in content
        model = genai.GenerativeModel('gemini-1.5-flash',system_instruction=user_input_with_context)
        chat = model.start_chat()
        directions="You are an evaluator. You need to determine if the following paragraphs are similar in content."
        directions += 'Paragraph 1:' + answer
        directions += 'Paragraph 2:' + 
        directions = 'Return 1 if they are similar, or 0 if they are not. Only return a 1 or 0.'
        res = chat.send_message(user_input)
        try:
            res = int(res)
        except:
            res = 0
        content_sim.append(res)
    # Calculate the average similarity
    avg_similarity = sum(scores) / len(scores) if scores else 0
    avg_similarity_content = sum(content_sim) / len(content_sim) if content_sim else 0
    return avg_similarity, avg_similarity_content

